## Travel Agent Tool

In [1]:
#!pip install langchain-prolog

from langchain_prolog import PrologConfig, PrologRunnable, PrologTool

In [2]:
schema = PrologRunnable.create_schema('query_route', ['query_type', 'from', 'to', 'options', 'results'])

config = PrologConfig(
            rules_file='routes.pl',
            query_schema=schema,
            default_predicate="query_route"
        )

planner_tool = PrologTool(
    prolog_config=config,
    name="travel_planner",
    description="""
        Query travel routes using Prolog.
        Input can be a query string like 'query_route(all, paris, london,[], Results)'.
        You have to specify 3 parameters:
            - query_type: can be 'all', 'fastest' or 'cheapest'
            - from: the city where the travel starts. Must be all lower case.
            - to: the city where the travel ends. Must be all lower case.
            - Results: This will be the key for the results
        The query will return:
            - 'False' if there are no routes availables
            - A dictionary with 'Results' as the key and a list ouf possible routes as the value
        Do not use quotes.
    """,
)

In [3]:
planner_tool.invoke(
    {
        'query_type': 'all', 
        'from': 'paris',
        'to': 'lisbon',
        'options': [],
        'results': None,
    }
)

[{'Results': [{'time': 11,
    'transport': [{'time': 2, 'type': 'plane', 'cost': 180},
     {'time': 2, 'type': 'plane', 'cost': 140},
     {'time': 3, 'type': 'train', 'cost': 80},
     {'time': 4, 'type': 'train', 'cost': 90}],
    'route': ['paris', 'rome', 'barcelona', 'madrid', 'lisbon'],
    'cost': 490},
   {'time': 8.5,
    'transport': [{'time': 1.5, 'type': 'plane', 'cost': 120},
     {'time': 3, 'type': 'train', 'cost': 80},
     {'time': 4, 'type': 'train', 'cost': 90}],
    'route': ['paris', 'barcelona', 'madrid', 'lisbon'],
    'cost': 290},
   {'time': 6,
    'transport': [{'time': 2, 'type': 'train', 'cost': 150},
     {'time': 4, 'type': 'train', 'cost': 90}],
    'route': ['paris', 'madrid', 'lisbon'],
    'cost': 240}]}]

### Using an LLM and function calling

In [4]:
#!pip install python-dotenv

from dotenv import find_dotenv
from dotenv import load_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [5]:
#!pip install langchain-openai

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools([planner_tool])

In [6]:
tries = 1
while tries <= 10:
    try:
        messages = [HumanMessage("I want to tavel from Paris to Lisbon. What are my options?")]
        response = llm_with_tools.invoke(messages)
        if hasattr(response, 'tool_calls'):
            messages.append(response)
            tool_msg = planner_tool.invoke(response.tool_calls[0])
            messages.append(tool_msg)
            response = llm_with_tools.invoke(messages)
        print(response.content)
        break
    except:
        tries += 1
        if tries > 10:
            print("Could not get an answer")

Here are the travel options from Paris to Lisbon:

1. **Route**: Paris → Rome → Barcelona → Madrid → Lisbon
   - **Total Travel Time**: 11 hours
   - **Transport Options**:
     - Plane: 2 hours (Cost: 180€)
     - Plane: 2 hours (Cost: 140€)
     - Train: 3 hours (Cost: 80€)
     - Train: 4 hours (Cost: 90€)
   - **Total Cost**: 490€

2. **Route**: Paris → Barcelona → Madrid → Lisbon
   - **Total Travel Time**: 8.5 hours
   - **Transport Options**:
     - Plane: 1.5 hours (Cost: 120€)
     - Train: 3 hours (Cost: 80€)
     - Train: 4 hours (Cost: 90€)
   - **Total Cost**: 290€

3. **Route**: Paris → Madrid → Lisbon
   - **Total Travel Time**: 6 hours
   - **Transport Options**:
     - Train: 2 hours (Cost: 150€)
     - Train: 4 hours (Cost: 90€)
   - **Total Cost**: 240€

You can choose the option that best suits your schedule and budget!


### Using an Agent

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor

llm = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [planner_tool]

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools)

In [8]:
tries = 1
while tries <= 10:
    try:
        results = agent_executor.invoke({"input": "I want to tavel from Paris to Lisbon. What are my options?"})
        print(results['output'])
        break
    except:
        tries += 1
        if tries > 10:
            print("Could not get an answer")

Here are your options for traveling from Paris to Lisbon:

1. **Route: Paris → Rome → Barcelona → Madrid → Lisbon**
   - **Total Travel Time:** 11 hours
   - **Transport Options:**
     - Plane: 2 hours, €180
     - Plane: 2 hours, €140
     - Train: 3 hours, €80
     - Train: 4 hours, €90
   - **Total Cost:** €490

2. **Route: Paris → Barcelona → Madrid → Lisbon**
   - **Total Travel Time:** 8.5 hours
   - **Transport Options:**
     - Plane: 1.5 hours, €120
     - Train: 3 hours, €80
     - Train: 4 hours, €90
   - **Total Cost:** €290

3. **Route: Paris → Madrid → Lisbon**
   - **Total Travel Time:** 6 hours
   - **Transport Options:**
     - Train: 2 hours, €150
     - Train: 4 hours, €90
   - **Total Cost:** €240

You can choose any of these routes based on your time and budget preferences!
